In [1]:
# setting my si data frame
import pandas as pd
data = '/workspaces/service-data/outputs/snapshots/2025-03-01/si.csv'
#si = pd.read_csv(data, sep=';')
si = pd.read_csv(data,
                  keep_default_na=False, 
                  na_values='', 
                  delimiter=';',
                 engine='python',
                skipfooter=2
                )
si_dp = si

from datetime import datetime
import pytz
import numpy as np

# Specify the timezone
timezone = pytz.timezone("America/Montreal")

# Get the current date and time in the specified timezone
current_date = datetime.now(timezone)

# Format the current date and time into the desired string format
current_datestr = current_date.strftime("%Y-%m-%d_%H:%M:%S")

# Print the current date and time
print(f"current date: {current_datestr}")

#si_dp = si #ge: Use si_dp in your calculations to keep a clean si for other stuff

# si = pd.read_csv("https://github.com/gcperformance/service-data/releases/latest/download/si.csv", 
#                  keep_default_na=False, 
#                  na_values='', 
#                  delimiter=';',
#                  engine='python',
#                  skipfooter=2
#                  )

current date: 2025-05-06_10:30:18


In [ ]:
#OMNICHANNEL FORMULA
#si_dp['omnichannel'] = si_dp.apply(
    #lambda row: (
        #pd.notna(pd.to_numeric(row['num_phone_enquiries'], errors='coerce') + 
                 #pd.to_numeric(row['num_applications_by_phone'], errors='coerce')) and
        #pd.notna(row['num_applications_online']) and
        #pd.notna(row['num_applications_in_person'])
    #),
    #axis=1
#)

si_dp['omnichannel'] = si_dp.apply(
    lambda row: (
        pd.notna(pd.to_numeric(row['num_phone_enquiries'], errors='coerce')) and
        pd.notna(pd.to_numeric(row['num_applications_by_phone'], errors='coerce')) and
        pd.notna(row['num_applications_online']) and
        pd.notna(row['num_applications_in_person'])
    ),
    axis=1
)


# Create 'phone_apps_enquiries' column (sum of phone enquiries + phone applications)

# Convert relevant columns to numeric and handle NaNs in the calculation
#si_dp['num_phone_enquiries'] = pd.to_numeric(si_dp['num_phone_enquiries'], errors='coerce').fillna(0)
#si_dp['num_applications_online'] = pd.to_numeric(si_dp['num_applications_online'], errors='coerce').fillna(0)
#si_dp['num_applications_in_person'] = pd.to_numeric(si_dp['num_applications_in_person'], errors='coerce').fillna(0)


si_dp['phone_apps_enquiries'] = (
    pd.to_numeric(si_dp['num_phone_enquiries'], errors='coerce').fillna(0) + 
    pd.to_numeric(si_dp['num_applications_by_phone'], errors='coerce').fillna(0)
)

# Create 'total_transactions' by summing all relevant application methods
si_dp['total_transactions'] = (
    pd.to_numeric(si_dp['num_applications_total'], errors='coerce').fillna(0) +
    pd.to_numeric(si_dp['num_phone_enquiries'], errors='coerce').fillna(0)
)

# adding a new column for applications done by phone, online and in person only
#si_dp['apps_online_and_per'] = (
    #pd.to_numeric(si['num_applications_in_person'], errors='coerce').fillna(0) + 
    #pd.to_numeric(si['num_applications_online'], errors='coerce').fillna(0) + 
    #pd.to_numeric(si['phone_apps_enquiries'], errors='coerce').fillna(0)
#)

# Ensure that relevant columns are numeric
#si_dp['phone_apps_enquiries'] = pd.to_numeric(si_dp['phone_apps_enquiries'], errors='coerce').fillna(0)
#si_dp['num_applications_online'] = pd.to_numeric(si_dp['num_applications_online'], errors='coerce').fillna(0)
#si_dp['num_applications_in_person'] = pd.to_numeric(si_dp['num_applications_in_person'], errors='coerce').fillna(0)

 
# adding a new column for external
si_dp['external'] = si_dp['service_scope'].str.contains('EXTERN', na=False)

# adding a new column for high volume services
si_dp['highvolume'] = si_dp['total_transactions'] >= 45000

# adding a new column for online enabled Y
# creating columns to check which lists out the columns from os_account_registration to os_issue_resolution_feedback
columns_to_check = [ 'os_account_registration', 'os_authentication', 'os_application', 'os_decision', 'os_issuance', 'os_issue_resolution_feedback']
si_dp['online_enabledY'] = si_dp[columns_to_check].apply(lambda row: (row == 'Y').sum(), axis=1)

# adding column for online enabled N
si_dp['online_enabledN'] = si_dp[columns_to_check].apply(lambda row: (row == 'N').sum(), axis=1)

# adding column for online enabled NA
si_dp['online_enabledNA'] = si_dp[columns_to_check].apply(lambda row: (row == 'NA').sum(), axis =1)
#si_dp['online_enabledNA'] = si_dp[columns_to_check].isna().sum(axis=1)

# adding a new column for online end to end
si_dp['onlineE2E'] = si_dp.apply(
    lambda row: (row['online_enabledNA'] + row['online_enabledY'] == 6) and (row['online_enabledY'] > 0),
    axis=1
)

# adding a new column for online one or more points
si_dp['oip'] = si_dp['online_enabledY'].apply(lambda x: True if x > 0 else False)

# Importing the service standards data
ss_data = '/workspaces/service-data/outputs/snapshots/2025-03-01/ss.csv' 
ss = pd.read_csv(ss_data, sep=';')
ss_dp = ss


In [6]:
# METRICs 2-16 - the transactions table will contain all the metrics that apply to external services alone

# Metric 2 - 3c
# Total number of transactions in millions
# online as a share of tatal transactions
# telephone as a share of total transactions
# in-person as a share of total transactions

# the metric total number of transactions as well as the channels as shares of total transactions may differ from last years service data pack analysis because of the methodology. 
# This year's analysis filters by external services filtered_si data frame filters all services to external
# Filter the si DataFrame where 'external' == 1
filtered_si = si_dp[si_dp['external'] == 1].copy() #GE: good idea to use copy, remember to use si_dp

# Ensure relevant columns are numeric
cols_to_numeric = [
    'total_transactions', 
    'num_applications_online', 
    'phone_apps_enquiries', 
    'num_applications_by_mail',
    'num_applications_in_person'
]
filtered_si[cols_to_numeric] = filtered_si[cols_to_numeric].apply(pd.to_numeric, errors='coerce').fillna(0)

# Group by fiscal_yr and sum the numeric columns
grouped = filtered_si.groupby('fiscal_yr')[cols_to_numeric].sum().reset_index()

# Rename columns
Transactions_table = grouped.rename(columns={
    'fiscal_yr': 'fiscal_year',
    'num_applications_online': 'online_applications',
    'phone_apps_enquiries': 'phone_applications',
    'num_applications_by_mail': 'mail_applications',
    'num_applications_in_person': 'in_person_apps'
})

# Add share columns
Transactions_table['online_share'] = (
    Transactions_table['online_applications'] / Transactions_table['total_transactions'] *100
).fillna(0)

Transactions_table['phone_share'] = (
    Transactions_table['phone_applications'] / Transactions_table['total_transactions'] * 100
).fillna(0)

Transactions_table['in_person_share'] = (
    Transactions_table['in_person_apps'] / Transactions_table['total_transactions'] *100
).fillna(0)

# METRIC 4-5C OMNICHANNEL OFFERINGS

# Ensure relevant columns in filtered_si are numeric
cols_omni = ['total_transactions', 'num_applications_online', 'phone_apps_enquiries', 'num_applications_in_person']
filtered_si[cols_omni] = filtered_si[cols_omni].apply(pd.to_numeric, errors='coerce').fillna(0)

# Count distinct omnichannel services per fiscal year
omni_count_by_year = filtered_si[filtered_si['omnichannel']].groupby('fiscal_yr')['service_id'].nunique()

# Count total distinct services per fiscal year
total_count_by_year = filtered_si.groupby('fiscal_yr')['service_id'].nunique()

# Calculate omnichannel service share
share_omni_by_year = (omni_count_by_year / total_count_by_year) * 100

# === Metric 5a: Online as a share of omnichannel usage ===
total_transactions_by_year_omni = filtered_si[filtered_si['omnichannel']].groupby('fiscal_yr')['total_transactions'].sum()
sum_online_apps_by_year = filtered_si[filtered_si['omnichannel']].groupby('fiscal_yr')['num_applications_online'].sum()
share_online_by_year = (sum_online_apps_by_year / total_transactions_by_year_omni) * 100

# === Metric 5b: Phone as a share of omnichannel usage ===
sum_phone_by_year = filtered_si[filtered_si['omnichannel']].groupby('fiscal_yr')['phone_apps_enquiries'].sum()
share_phone_by_year = (sum_phone_by_year / total_transactions_by_year_omni) * 100

# === Metric 5c: In-person as a share of omnichannel usage ===
sum_in_person_by_year = filtered_si[filtered_si['omnichannel']].groupby('fiscal_yr')['num_applications_in_person'].sum()
share_in_person_by_year = (sum_in_person_by_year / total_transactions_by_year_omni) * 100

# === Reindex for consistent fiscal years ===
all_fiscal_years = Transactions_table['fiscal_year']

Transactions_table['omnichannel_service_count'] = omni_count_by_year.reindex(all_fiscal_years).values
Transactions_table['total_service_count'] = total_count_by_year.reindex(all_fiscal_years).values
Transactions_table['gc_services_with_omnichannel'] = share_omni_by_year.reindex(all_fiscal_years).values

Transactions_table['total_transactions_omnichannel'] = total_transactions_by_year_omni.reindex(all_fiscal_years).values
Transactions_table['omnichannel_online_apps'] = sum_online_apps_by_year.reindex(all_fiscal_years).values
Transactions_table['online_share_of_omnichannel_usage'] = share_online_by_year.reindex(all_fiscal_years).values

Transactions_table['omnichannel_phone_apps'] = sum_phone_by_year.reindex(all_fiscal_years).values
Transactions_table['phone_share_of_omnichannel_usage'] = share_phone_by_year.reindex(all_fiscal_years).values

Transactions_table['omnichannel_inperson_apps'] = sum_in_person_by_year.reindex(all_fiscal_years).values
Transactions_table['in-person_share_of_omnichannel_usage'] = share_in_person_by_year.reindex(all_fiscal_years).values

# METRIC 6-9 NUMBER OF External departments, programs, services, and high volume services
# METRIC 6: Number of External Departments 
no_departments_by_year = filtered_si.groupby('fiscal_yr')['department_en'].nunique()
no_departments_by_year = no_departments_by_year.reindex(Transactions_table['fiscal_year']).values
Transactions_table['external_departments'] = no_departments_by_year

# METRIC 7: Number of Programs 

# Extract and clean program_id entries
no_program = filtered_si.loc[:, ['service_id', 'fiscal_yr', 'program_id', 'org_id']].copy()
no_program['program_id'] = no_program['program_id'].astype(str).str.split(',')  # ensure it's a string before split
no_program = no_program.explode('program_id')
no_program = no_program[no_program['program_id'].notna()]
no_program['program_id'] = no_program['program_id'].str.strip()  # remove leading/trailing spaces

# Count unique program_id per fiscal year
no_programs_by_year = no_program.groupby('fiscal_yr')['program_id'].nunique()

# Add to Transactions_table using reindex to align fiscal years
Transactions_table['external_programs'] = no_programs_by_year.reindex(Transactions_table['fiscal_year']).values

# METRIC 8: Number of External Services 
no_external_service_by_year = filtered_si.groupby('fiscal_yr')['service_id'].nunique()
no_external_service_by_year = no_external_service_by_year.reindex(Transactions_table['fiscal_year']).values
Transactions_table['external_services'] = no_external_service_by_year

# METRIC 9: Number of High Volume Services 

high_volume_services = filtered_si[filtered_si['highvolume'] == True]
high_volume_by_year = high_volume_services.groupby('fiscal_yr')['service_id'].nunique()
Transactions_table['high_volume_services'] = high_volume_by_year.reindex(Transactions_table['fiscal_year']).values

# METRIC 10-13 - total online, phone, in-person and mail transactions in millions
# the columns with this information already exist from metric 2-3c.

# METRIC 14-15 SHARE of external services online end-to-end and share of external services with at least one point online
# Metric 14: share of external services online end-to-end
# NUMERATOR:  Count unique service_id per fiscal year where onlineE2E is True
services_onlinee2e = (
    filtered_si.loc[filtered_si['onlineE2E'] == True]
    .groupby('fiscal_yr')['service_id']
    .nunique()
)

# Align with Transactions_table fiscal years
services_onlinee2e = services_onlinee2e.reindex(Transactions_table['fiscal_year'])
# Add the result to Transactions_table
Transactions_table['services_onlinee2e'] = services_onlinee2e.values

#SHARE OF EXTERNAL SERVICES ONLINE END TO END
# the methodology also differs from last years analysis. previously, the DENOMINATOR was external_services == TRUE and online_enabledNA < 6. This years methodology is external_services == TRUE ONLY
# Calculate the percentage of services that are online end-to-end
Transactions_table['share_of_services_online_end_to_end'] = (
    Transactions_table['services_onlinee2e'] / Transactions_table['external_services']
) * 100

# Fill NaN values with 0 to handle division by zero or missing data
Transactions_table['share_of_services_online_end_to_end'] = Transactions_table['share_of_services_online_end_to_end'].fillna(0)

#Metric 15: Share of external services which have at least one point online

# NUMERATOR: Services with at least one point online
# Filter rows where oip is True and count unique service_id per fiscal year
services_with_oip = (
    filtered_si[filtered_si['oip'] == True]
    .groupby('fiscal_yr')['service_id']
    .nunique()
)
# Align with Transactions_table fiscal years
services_with_oip = services_with_oip.reindex(Transactions_table['fiscal_year'])
Transactions_table['services_with_oip'] = services_with_oip.values
Transactions_table['share_services_with_oip'] = (
    Transactions_table['services_with_oip'] / Transactions_table['external_services']
) * 100

# METRIC 16: External service standards meeting targets
# from Process script MAF 2 line 225
# What is the percentage of service standards that met their target?
# The methodology differs from last year. last years: counts if external service is TRUE and met at least one standard / external service = TRUE
# current methodology counts the target met == Y / service standard count

# Select relevant columns and drop rows with missing values
standard_met = ss_dp.loc[:, ['fiscal_yr', 'service_standard_id', 'department_en', 'department_fr', 'org_id', 'target_met']].dropna()

# Check if the service standard target is met
standard_met['service_standard_met'] = standard_met['target_met'] == 'Y'

# Count service standards that have a target met (Y) or not met (N)
standard_met['service_standard_count'] = standard_met['target_met'].isin(['Y', 'N'])

# Group by fiscal year and aggregate the count of service standards
standard_met_by_fiscal = standard_met.groupby(['fiscal_yr']).agg(
    service_standard_met=('service_standard_met', 'sum'),  # Count services where target was met
    service_standard_count=('service_standard_count', 'sum')  # Total service standards
).reset_index()

# Calculate the percentage of standards that met the target for each fiscal year
standard_met_by_fiscal['standard_meeting_target'] = (
    standard_met_by_fiscal['service_standard_met'] / standard_met_by_fiscal['service_standard_count']
) * 100

# Align with Transactions_table fiscal years
standard_met_by_fiscal = standard_met_by_fiscal.set_index('fiscal_yr')

# Add the result to Transactions_table
Transactions_table['service_standard_met'] = standard_met_by_fiscal['service_standard_met'].reindex(Transactions_table['fiscal_year']).values
Transactions_table['service_standard_count'] = standard_met_by_fiscal['service_standard_count'].reindex(Transactions_table['fiscal_year']).values
Transactions_table['service_standard_target_met'] = standard_met_by_fiscal['standard_meeting_target'].reindex(Transactions_table['fiscal_year']).values

# METRIC 17 - 19 External and high volume services
# External high volume services online end-to-end, that have atleast one point online and that meet their targets
# The methodology for this year differs from last year
# same methodology for metric 14 and 16 is applied here but in addition highvolume == TRUE

# Metric 17: external high volume services online end to end
# NUMERATOR: Count unique service_id per fiscal year where onlineE2E and highvolume are both True
services_onlinee2e_highvolume = (
    filtered_si.loc[(filtered_si['onlineE2E'] == True) & (filtered_si['highvolume'] == True)]
    .groupby('fiscal_yr')['service_id']
    .nunique()
)
# Align with Transactions_table fiscal years
services_onlinee2e_highvolume = services_onlinee2e_highvolume.reindex(Transactions_table['fiscal_year'])
# Add the result to Transactions_table
Transactions_table['service_onlinee2e_highvolume'] = services_onlinee2e_highvolume.values

#SHARE OF EXTERNAL SERVICES ONLINE END TO END
# Calculate the percentage of services that are online end-to-end
Transactions_table['share_of_highvol_ser_online_end_to_end'] = (
    Transactions_table['service_onlinee2e_highvolume'] / Transactions_table['high_volume_services']
) * 100

# Fill NaN values with 0 to handle division by zero or missing data
Transactions_table['share_of_highvol_ser_online_end_to_end'] = Transactions_table['share_of_highvol_ser_online_end_to_end'].fillna(0)

# Metric 18: High volume services that have at least one point online
# NUMERATOR: High-volume services with at least one point online
# Filter rows where oip is True and highvolume is True, then count unique service_id per fiscal year
highvol_ser_with_oip = (
    filtered_si[(filtered_si['oip'] == True) & (filtered_si['highvolume'] == True)]
    .groupby('fiscal_yr')['service_id']
    .nunique()
)

# Align with Transactions_table fiscal years
highvol_ser_with_oip = highvol_ser_with_oip.reindex(Transactions_table['fiscal_year'])

# Add the result to Transactions_table
Transactions_table['highvol_service_with_at_least_oip'] = highvol_ser_with_oip.values

# SHARE OF HIGH-VOLUME SERVICES WITH AT LEAST ONE ONLINE INTERACTION POINT
Transactions_table['share_highvolume_services_with_oip'] = (
    Transactions_table['highvol_service_with_at_least_oip'] / Transactions_table['high_volume_services']
) * 100

# METRIC 19: High volume service standards meeting targets

# Select relevant columns from ss_dp and drop rows with missing values
highvol_standard_met = ss_dp.loc[:, ['fiscal_yr', 'service_standard_id', 'department_en', 'department_fr',
                             'org_id', 'target_met', 'fy_org_id_service_id']].dropna()

# Merge with si_dp to get 'highvolume' column
highvol_standard_met = highvol_standard_met.merge(
    si_dp[['fy_org_id_service_id', 'highvolume']],
    on='fy_org_id_service_id',
    how='left'  # Use 'left' to preserve all records from ss_dp
)

# Filter only high volume service standards
highvol_standard_met = highvol_standard_met[highvol_standard_met['highvolume'] == True]

# Check if the service standard target is met
highvol_standard_met['highvol_service_standard_met'] = highvol_standard_met['target_met'] == 'Y'

# Count service standards that have a target met (Y) or not met (N)
highvol_standard_met['highvol_service_standard_count'] = highvol_standard_met['target_met'].isin(['Y', 'N'])

# Group by fiscal year and aggregate the count of service standards
highvol_standard_met_by_fiscal = highvol_standard_met.groupby(['fiscal_yr']).agg(
    highvol_service_standard_met=('highvol_service_standard_met', 'sum'),  # Count high volume services where target was met
    highvol_service_standard_count=('highvol_service_standard_count', 'sum')  # Total high volume service standards
).reset_index()

# Calculate the percentage of high volume standards that met the target for each fiscal year
highvol_standard_met_by_fiscal['highvol_standard_meeting_target'] = (
    highvol_standard_met_by_fiscal['highvol_service_standard_met'] / highvol_standard_met_by_fiscal['highvol_service_standard_count']
) * 100

# Align with Transactions_table fiscal years
highvol_standard_met_by_fiscal = highvol_standard_met_by_fiscal.set_index('fiscal_yr')

# Add the result to Transactions_table
Transactions_table['highvol_service_standard_met'] = highvol_standard_met_by_fiscal['highvol_service_standard_met'].reindex(Transactions_table['fiscal_year']).values
Transactions_table['highvol_service_standard_count'] = highvol_standard_met_by_fiscal['highvol_service_standard_count'].reindex(Transactions_table['fiscal_year']).values
Transactions_table['pc_highvol_service_standard_target_met'] = highvol_standard_met_by_fiscal['highvol_standard_meeting_target'].reindex(Transactions_table['fiscal_year']).values




In [7]:
# Final output
# print(Transactions_table)

# Export the Transactions_table to a CSV file
Transactions_table.to_csv('Transactions_table.csv', index=True)

# Transpose the Transactions_table such that fiscal_year becomes the columns
Transactions_table_transposed = Transactions_table.set_index('fiscal_year').transpose()

# Final output: print the transposed table
# print(Transactions_table_transposed)

# Export the transposed Transactions_table to a CSV file
Transactions_table_transposed.to_csv('Transactions_table_transposed.csv', index=True)

In [7]:
Transactions_table

,fiscal_year,total_transactions,online_applications,phone_applications,mail_applications,in_person_apps,online_share,phone_share,in_person_share,omnichannel_service_count,...,service_standard_met,service_standard_count,service_standard_target_met,service_onlinee2e_highvolume,share_of_highvol_ser_online_end_to_end,highvol_service_with_at_least_oip,share_highvolume_services_with_oip,highvol_service_standard_met,highvol_service_standard_count,pc_highvol_service_standard_target_met
0,2018-2019,421133885.0,174630378.0,53099291.0,42316481.0,118928362.0,41.466713,12.608648,28.240036,1130,...,876.0,1295.0,67.644788,32.0,27.350427,84.0,71.794872,105.0,173.0,60.693642
1,2019-2020,412398882.0,156831471.0,90704008.0,33221419.0,111052720.0,38.029073,21.994242,26.928473,1179,...,990.0,1401.0,70.663812,44.0,34.920635,96.0,76.190476,146.0,207.0,70.531401
2,2020-2021,386183263.0,202706431.0,137242208.0,19468112.0,15198693.0,52.489699,35.538104,3.935617,1304,...,1292.0,1806.0,71.539313,59.0,47.967480,95.0,77.235772,148.0,291.0,50.859107
3,2021-2022,358629620.0,196235510.0,104413441.0,22377948.0,27955327.0,54.718155,29.114561,7.795041,112,...,1242.0,1852.0,67.062635,63.0,47.014925,106.0,79.104478,111.0,230.0,48.260870
4,2022-2023,464461678.0,268647250.0,72927098.0,31707932.0,78959836.0,57.840563,15.701424,17.000291,86,...,1378.0,2020.0,68.217822,50.0,43.103448,89.0,76.724138,197.0,300.0,65.666667
5,2023-2024,462846031.0,254970346.0,71119791.0,21405253.0,102586170.0,55.087508,15.365756,22.164211,642,...,1295.0,1765.0,73.371105,55.0,47.008547,88.0,75.213675,200.0,301.0,66.445183
6,2024-2025,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
